## Concurrency

In [ ]:
# We all know that concurrent programming including programming with threads, launching subprocesses and various tricks involving generator functions.




: 

## Starting and Stopping Threads

In [18]:
# The threading library can be used to execute any python callable in its own thread. To do this you create a thread instance and supply the callable that you wish to execute as a target.

# Cod to execute in an independent thread
import time

def countdown(n):
    while n>0:
        print('T minus', n)
        n -= 1
        time.sleep(5)

# Create and launch a thread

from threading import Thread

t = Thread(target=countdown, args=(10,))
t.start()

T minus 10


T minus 9


In [16]:
if t.is_alive():
    print('Still Running')
else:
    print('Completed')

Completed


In [19]:
# For long  running threads or if we want the threads to run forever we should consider making the thread daemonic


t = Thread(target = countdown, args=(10,), daemon=True)
t.start()

T minus 10


T minus 8
T minus 9
T minus 7
T minus 8
T minus 6
T minus 7
T minus 5
T minus 6
T minus 4
T minus 5
T minus 3
T minus 4
T minus 2
T minus 3
T minus 1
T minus 2
T minus 1


## The following [20,21,21] cells describe how to terminate a thread

In [20]:
# If you want to be able to terminate threads, the thread must be programmed to poll for exit at selected points. For example, you might put your thread in a class such  as this:

class CountdownTask:
    def __init__(self):
        self._running = True

    def terminate(self):
        self._running = False
    
    def run(self, n):
        while self._running and n>0:
            print('T-minus',n)
            n -= 1
            time.sleep(3)

c = CountdownTask()
t = Thread(target=c.run, args=(10,), daemon=True)

In [21]:
t.start()

T-minus 10


T-minus 9
T-minus 8


In [22]:
c.terminate()